#### Installation Of All Required Dependecies

In [3]:
!conda install pandas --yes
!conda install numpy --yes
!conda install xarray --yes
!conda install -c anaconda netcdf4 --yes
!pip install matplotlib --yes

#### Importing Of All Required Dependecies

In [21]:
import numpy as np
import pandas as pd
import xarray as xr
from netCDF4 import Dataset
import os,datetime,sys,fnmatch

In [22]:
myd03_name = '/Users/saviosebastian/Documents/Project/CMAC/Files/Up/MYD03.A2008001.0000.006.2012066122450.hdf'
myd06_name = '/Users/saviosebastian/Documents/Project/CMAC/Files/MYD06_L2.A2008013.0405.006.2013342064735.hdf'

myd03 = Dataset(myd03_name, "r")
latitude = myd03.variables["Latitude"][:,:] # Reading Specific Variable 'Latitude'.
latitude = np.ravel(latitude) # Changing The Shape Of The Variable 'Latitude'.
latitude = np.array(latitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.

longitude = myd03.variables["Longitude"][:,:] # Reading Specific Variable 'Longitude'.
longitude = np.ravel(longitude) # Changing The Shape Of The Variable 'Longitude'.
longitude = np.array(longitude).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.

myd06 = Dataset(myd06_name, "r")
CM = myd06.variables["Cloud_Mask_1km"][:,:,:] # Reading Specific Variable 'Cloud_Mask_1km'.
CM_bit0 = np.array(CM[:,:,0].data,dtype='byte') & 0b00000001
CM_bit12 = (np.array(CM[:,:,0].data,dtype='byte') & 0b00000110) >> 1
CMr = np.ravel(CM) # Changing The Shape Of The Variable 'Cloud_Mask_1km'.
CMr = np.array(CMr).byteswap().newbyteorder() # Addressing Byteswap For Big Endian Error.
CMg = np.ravel(CM_bit0)
CMB12 = np.ravel((np.array(CM[:,:,0].data,dtype='byte') & 0b00000110) >> 1)

#### Attempting To Read Multiple Files 

In [13]:
def read_MODIS_level2_data(MOD06_file,MOD03_file):
    print(MOD06_file)
    print(MOD03_file)
    print('reading the cloud mask from MOD06_L2 product')
    MOD06 = Dataset(MOD06_file, "r")
    CM1km = MOD06.variables["Cloud_Mask_1km"][:,:,:]
    CM   = (np.array(CM1km[:,:,0],dtype='byte') & 0b00000110) >>1
    print('level-2 cloud mask array shape',CM.shape)

    MOD03 = Dataset(MOD03_file,'r')
    print('reading the lat-lon from MOD03 product')
    lat = MOD03.variables["Latitude"][:,:]
    lon = MOD03.variables["Longitude"][:,:]
    print('level-2 lat-lon array shape',lat.shape)

    return lat,lon,CM

In [18]:
MOD03_fp = 'MYD03.A*.hdf'
MOD06_fp = 'MYD06_L2.A*.hdf'
MOD03_fn, MOD06_fn =[],[]
#Declare Xarrya and append to it in the for loop
for MOD06_flist in  os.listdir(MOD06_path):
    if fnmatch.fnmatch(MOD06_flist, MOD06_fp):
           MOD06_fn = MOD06_flist
    for MOD03_flist in  os.listdir(MOD03_path):
        if fnmatch.fnmatch(MOD03_flist, MOD03_fp):
           MOD03_fn = MOD03_flist
    if MOD03_fn and MOD06_fn: # if both MOD06 and MOD03 products are in the directory
        print('reading level 2 geolocation and cloud data')
        print(MOD06_fn)
        Lat,Lon,CM = read_MODIS_level2_data(MOD06_path+MOD06_fn,MOD03_path+MOD03_fn)
        Lat=np.ravel(Lat)
        Lon=np.ravel(Lon)
        CM=np.ravel(CM)
        #Append the CM TO THe Xarray.
        print(CM)

reading level 2 geolocation and cloud data
MYD06_L2.A2016008.0000.061.2018059051528.hdf
/Users/saviosebastian/Documents/Project/CMAC/Files/MYD06_L2.A2016008.0000.061.2018059051528.hdf
/Users/saviosebastian/Documents/Project/CMAC/File2/MYD03.A2008001.0000.006.2012066122450.hdf
reading the cloud mask from MOD06_L2 product
level-2 cloud mask array shape (2030, 1354)
reading the lat-lon from MOD03 product
level-2 lat-lon array shape (2030, 1354)
[2 2 2 ... 3 3 3]
reading level 2 geolocation and cloud data
MYD06_L2.A2008013.0405.006.2013342064735.hdf
/Users/saviosebastian/Documents/Project/CMAC/Files/MYD06_L2.A2008013.0405.006.2013342064735.hdf
/Users/saviosebastian/Documents/Project/CMAC/File2/MYD03.A2008001.0000.006.2012066122450.hdf
reading the cloud mask from MOD06_L2 product
level-2 cloud mask array shape (2030, 1354)
reading the lat-lon from MOD03 product
level-2 lat-lon array shape (2030, 1354)
[0 0 0 ... 0 0 0]
reading level 2 geolocation and cloud data
MYD06_L2.A2008013.0405.006.20

In [19]:
MOD06_fn

'MYD06_L2.A2008013.0440.006.2013342071607.hdf'

In [23]:
import itertools
MOD03_path = '/Users/saviosebastian/Documents/Project/CMAC/File2/'
MOD06_path = '/Users/saviosebastian/Documents/Project/CMAC/Files/'
satellite = 'Aqua'

In [24]:
print("Bit 0 - Cloud Mask Flag")
print("Flag 0 : Not Determined | Flag 1 : Determined ")
print("----------------------------------------------")
print("Minimum Of CM_bit0: ", CM_bit0.min(), " | Maximum Of CM_bit0: ",CM_bit0.max())

Bit 0 - Cloud Mask Flag
Flag 0 : Not Determined | Flag 1 : Determined 
----------------------------------------------
Minimum Of CM_bit0:  1  | Maximum Of CM_bit0:  1


In [25]:
print("Bit 12 - Unobstructed FOV Quality Flag: ")
print("00 : Cloudy(0) & 01 : Uncertain Clear(1)")
print("& 10 : Probably Clear(2) & 11 : Confident Clear(3)")
print("--------------------------------------------------")
print("Unique Set Values Of CM_bit12: ",np.unique(CM_bit12))

Bit 12 - Unobstructed FOV Quality Flag: 
00 : Cloudy(0) & 01 : Uncertain Clear(1)
& 10 : Probably Clear(2) & 11 : Confident Clear(3)
--------------------------------------------------
Unique Set Values Of CM_bit12:  [0 1 2 3]


In [26]:
Nobs = np.sum(CM_bit0>=0)
print("Number Of Total Pixel: ", np.sum(CM_bit0>=0), " Pixels")
print("Number Of Pixel With Cloud Mask Determined: ", np.sum(CM_bit0==1), " Pixels")

Number Of Total Pixel:  2748620  Pixels
Number Of Pixel With Cloud Mask Determined:  2748620  Pixels


In [27]:
CM_types = {0:'Cloudy Pixels', 1:'Uncertain Clear Pixels', 2:'Probably Clear Pixels', 3:'Confident Clear Pixels'}
for key, value in CM_types.items():
    print("Number Of "+value+" : ",np.sum((CM_bit0==1) & (CM_bit12==key)),np.sum((CM_bit0==1) & (CM_bit12==key))/Nobs*100,'%')

Number Of Cloudy Pixels :  1535673 55.87069147426709 %
Number Of Uncertain Clear Pixels :  295661 10.756707002059215 %
Number Of Probably Clear Pixels :  163662 5.954333447329932 %
Number Of Confident Clear Pixels :  753624 27.418268076343765 %


#### GroupBy Implementation With Pandas

In [28]:
d = {'Latitude' :  pd.Series(latitude), 'Longitude' : pd.Series(longitude), 'Bit 0' :  pd.Series(CMg), 'Bit 12' : pd.Series(CMB12)}
df1 = pd.DataFrame(d,columns=['Latitude', 'Longitude', 'Bit 0', 'Bit 12'])

In [29]:
df1

,Latitude,Longitude,Bit 0,Bit 12
0,69.990623,15.142107,1,0
1,69.994247,15.270035,1,0
2,69.997742,15.397042,1,0
3,70.001122,15.523101,1,0
4,70.004395,15.648222,1,0
5,70.007545,15.772418,1,0
6,70.010597,15.895703,1,1
7,70.013535,16.018047,1,1
8,70.016365,16.139532,1,1
9,70.019096,16.260153,1,0


In [134]:
print(list(dict(df1.groupby(['Bit 0','Bit 12']).groups)))
df1.groupby(['Bit 0','Bit 12']).groups

[(1, 0), (1, 1), (1, 2), (1, 3)]


{(1,
  0): Int64Index([      0,       1,       2,       3,       4,       5,       9,
                  10,      11,      12,
             ...
             2748610, 2748611, 2748612, 2748613, 2748614, 2748615, 2748616,
             2748617, 2748618, 2748619],
            dtype='int64', length=1535673),
 (1,
  1): Int64Index([      6,       7,       8,      14,      15,      16,      20,
                  32,      36,      38,
             ...
             2747862, 2747867, 2747871, 2747883, 2747899, 2747941, 2747995,
             2748590, 2748592, 2748598],
            dtype='int64', length=295661),
 (1,
  2): Int64Index([     17,      19,      33,      45,      46,      48,      49,
                  50,      51,      52,
             ...
             2747979, 2747980, 2747981, 2747983, 2748000, 2748001, 2748009,
             2748052, 2748053, 2748591],
            dtype='int64', length=163662),
 (1,
  3): Int64Index([     18,      37,      47,      54,      55,      56,      58,
    

In [137]:
CM_types = {0:'Cloudy Pixels', 1:'Uncertain Clear Pixels', 2:'Probably Clear Pixels', 3:'Confident Clear Pixels'}
print(CM_types)
df1.groupby(['Bit 0','Bit 12']).count()

{0: 'Cloudy Pixels', 1: 'Uncertain Clear Pixels', 2: 'Probably Clear Pixels', 3: 'Confident Clear Pixels'}


Latitude  Longitude
Bit 0 Bit 12                     
1     0        1535673    1535673
      1         295661     295661
      2         163662     163662
      3         753624     753624

In [123]:
CM_types = {0:'Cloudy Pixels', 1:'Uncertain Clear Pixels', 2:'Probably Clear Pixels', 3:'Confident Clear Pixels'}
print(CM_types)
df1.groupby(['Bit 0','Bit 12']).count()/Nobs*100

{0: 'Cloudy Pixels', 1: 'Uncertain Clear Pixels', 2: 'Probably Clear Pixels', 3: 'Confident Clear Pixels'}


Latitude  Longitude
Bit 0 Bit 12                      
1     0       55.870691  55.870691
      1       10.756707  10.756707
      2        5.954333   5.954333
      3       27.418268  27.418268

#### GroupBy Implementation With XArrays

In [178]:
dsa = xr.Dataset()
dsa['Latitude'] = (('x'), latitude)
dsa['Longitude'] = (('x'), longitude)
dsa.coords['Bit0'] = (('x'), CMg)
dsa.coords['Bit12'] = (('x'), CMB12)

In [179]:
dsa

<xarray.Dataset>
Dimensions:    (x: 2748620)
Coordinates:
    Bit0       (x) int8 1 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1 1
    Bit12      (x) int8 0 0 0 0 0 0 1 1 1 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0
Dimensions without coordinates: x
Data variables:
    Latitude   (x) float32 69.99062 69.99425 69.99774 ... 47.469513 47.45127
    Longitude  (x) float32 15.142107 15.270035 15.397042 ... 47.72328 47.78136

In [180]:
list(dsa.groupby('Bit12','Bit0'))

[(0, <xarray.Dataset>
  Dimensions:    (x: 1535673)
  Coordinates:
      Bit0       (x) int8 1 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1 1
      Bit12      (x) int8 0 0 0 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0 0 0 0 0
  Dimensions without coordinates: x
  Data variables:
      Latitude   (x) float32 69.99062 69.99425 69.99774 ... 47.469513 47.45127
      Longitude  (x) float32 15.142107 15.270035 15.397042 ... 47.72328 47.78136),
 (1, <xarray.Dataset>
  Dimensions:    (x: 295661)
  Coordinates:
      Bit0       (x) int8 1 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1 1
      Bit12      (x) int8 1 1 1 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1 1 1 1
  Dimensions without coordinates: x
  Data variables:
      Latitude   (x) float32 70.0106 70.013535 70.016365 ... 47.890717 47.801613
      Longitude  (x) float32 15.895703 16.018047 16.139532 ... 46.34328 46.64171),
 (2, <xarray.Dataset>
  Dimensions:    (x: 163662)
  Coordinates:
      Bit0       (x) int8 1 1 1 1 1 1 1

In [183]:
CM_types = {0:'Cloudy Pixels', 1:'Uncertain Clear Pixels', 2:'Probably Clear Pixels', 3:'Confident Clear Pixels'}
print(CM_types)
dsa.groupby('Bit12','Bit0').count()

{0: 'Cloudy Pixels', 1: 'Uncertain Clear Pixels', 2: 'Probably Clear Pixels', 3: 'Confident Clear Pixels'}


<xarray.Dataset>
Dimensions:    (Bit12: 4)
Coordinates:
  * Bit12      (Bit12) int64 0 1 2 3
Data variables:
    Latitude   (Bit12) int64 1535673 295661 163662 753624
    Longitude  (Bit12) int64 1535673 295661 163662 753624

In [184]:
CM_types = {0:'Cloudy Pixels', 1:'Uncertain Clear Pixels', 2:'Probably Clear Pixels', 3:'Confident Clear Pixels'}
print(CM_types)
dsa.groupby('Bit12','Bit0').count()/Nobs*100

{0: 'Cloudy Pixels', 1: 'Uncertain Clear Pixels', 2: 'Probably Clear Pixels', 3: 'Confident Clear Pixels'}


<xarray.Dataset>
Dimensions:    (Bit12: 4)
Coordinates:
  * Bit12      (Bit12) int64 0 1 2 3
Data variables:
    Latitude   (Bit12) float64 55.87 10.76 5.954 27.42
    Longitude  (Bit12) float64 55.87 10.76 5.954 27.42